In [1]:
from sklearn.model_selection import train_test_split
from sklearn import svm
from skimage import feature
import numpy as np
from skimage.transform import rotate
from skimage.feature import local_binary_pattern
from skimage import data
from skimage.color import label2rgb
import matplotlib.pyplot as plt
import numpy as np
import cv2
from os import listdir
import time
from os.path import isfile, join


# Load dataset
dataset = np.loadtxt('base1200maiz.txt', delimiter='\t')
data, labels = dataset[:, 0:4], dataset[:, 5]

# Split dataset
data_train, data_test, labels_train, labels_test = train_test_split(data, labels, test_size=0.18, random_state=42)

In [2]:
print(data_train)

[[0.33   0.2075 0.225  0.2375]
 [0.24   0.3625 0.205  0.1925]
 [0.2    0.3475 0.295  0.1575]
 ...
 [0.2475 0.2825 0.285  0.185 ]
 [0.2825 0.255  0.2475 0.215 ]
 [0.1575 0.415  0.2975 0.13  ]]


In [3]:
data_train.shape

(984, 4)

In [4]:
# Train classifier
clf = svm.SVC(kernel='rbf')
clf.fit(data_train, labels_train)
print("Porcentaje de efectividad: ", clf.score(data_test, labels_test))
features = []

class LocalBinaryPatterns:
	def __init__(self, numPoints, radius):
		# store the number of points and radius
		self.numPoints = numPoints
		self.radius = radius
	def describe(self, image, eps=1e-7):
		# compute the Local Binary Pattern representation
		# of the image, and then use the LBP representation
		# to build the histogram of patterns
		lbp = feature.local_binary_pattern(image, self.numPoints,
			self.radius, method="uniform")
		(hist, _) = np.histogram(lbp.ravel(),
			bins=np.arange(0, self.numPoints + 3),
			range=(0, self.numPoints + 2))
		# normalize the histogram
		hist = hist.astype("float")
		hist /= (hist.sum() + eps)
		# return the histogram of Local Binary Patterns
		return hist


Porcentaje de efectividad:  0.4722222222222222


In [5]:
img = cv2.imread(r'C:\Users\Sebastian\OneDrive - INSTITUTO TECNOLOGICO METROPOLITANO - ITM\Desktop\Cursos_vision artificial\Clasificacion_Enfermedades_Hojas_Maiz\data\Gray_Leaf_Spot\Corn_Gray_Spot (6).jpg')
img3 = cv2.imread(r'C:\Users\Sebastian\OneDrive - INSTITUTO TECNOLOGICO METROPOLITANO - ITM\Desktop\Cursos_vision artificial\Clasificacion_Enfermedades_Hojas_Maiz\data\Gray_Leaf_Spot\Corn_Gray_Spot (6).jpg')
print(img.shape)
height, width,profun = img3.shape[:]  # Obtenemos sus dimensiones
img2 = np.zeros((height, width,profun), np.uint8) #creamos una imagen vacia

(433, 527, 3)


In [6]:
#  funcion de llamado del mouse
def draw_rec(event, x, y, flags, param): # Se declara la funcion. 
    global ix, iy, drawing, mode,x2,y2,x1,y1  # Defino unas variables globales
    if event == cv2.EVENT_LBUTTONDOWN:  # Se pregunta si se ha presionado el mouse,tecla izqu
        drawing = True  # En caso de ser verdado se asigna una variable boleana
        ix, iy = x, y  # Almacenamos la posicion incial en las variales
        x1,y1=x,y# variables iniciales para el recorte de la imagen
        print(ix, iy)
        for i in range(0,21):
                  x=ix+i
                  y=iy+i
                  cv2.rectangle(img, (ix-20, iy-20), (x,y), (0, 0, 0), -1)

        x2,y2 = x,y #coordenadas rectangulo
                 
cv2.namedWindow('image') #nombre de la ventana
cv2.setMouseCallback('image', draw_rec)  # Muestro las imagenes, operar con eventos por medio del mouse sobre la ventana creada,draw circle recibe los eventos

In [7]:
while 1:#while true, ciclo infinito
    if cv2.waitKey(1) & 0xFF == ord('c'):
        break 
    
    cv2.imshow('image', img)#muestra la imagen creada en la ventana
    if cv2.waitKey(1) & 0xFF == ord('q'):#actualiza constantemente la imagen,si presiona 'q' para cerrar
        if x2 > height:#restringimos las coordenadas del rectangulo
            x2 = height
        elif y2 > width:
            y2 = width
        
        for j in range(x1, x2):
            for i in range(y1, y2):
                img2[i,j,:] = img3[i,j,:] #Guardamos el rectangulo en una nueva imagen con sus respectivas coor
                
        imgcropped=img3[iy:y2,ix:x2]
        gray=cv2.cvtColor(imgcropped, cv2.COLOR_BGR2GRAY)
        descriptor=LocalBinaryPatterns(3,1)
        histograma=descriptor.describe(gray)
        histogramat = histograma.astype("float")
        features = np.array([histograma[0],histograma[1],histograma[2],histograma[3]])
        features=features.reshape(1, -1)
       # print(features.shape)
        enf = clf.predict(features)
        print(enf)

        if enf == 0:
            print("Hoja Sana")
        elif enf == 1:
            print("Enfermedad de la hoja gris")
        elif enf == 2:
            print("Enfermedad del óxido")
        elif enf == 3:
            print("Enfermedad por plaga")
       
        
        cv2.imshow('img',img)
        cv2.imshow('recorte',imgcropped)
        cv2.waitKey(0)
        cv2.destroyAllWindows()      
                       
        # Se esperan 30ms para el cierre de la ventana o hasta que el usuario presione la tecla q
cv2.waitKey(0)  # Se espera a pulsar cualquier tecla para cerrar la imagen
cv2.destroyAllWindows()  # Cierre de ventanas

225 101
[0.]
Hoja Sana


KeyboardInterrupt: 

: 